## 1) Prerequisiti (breve)
- Raspberry Pi con Raspbian / Raspberry OS e rete accessibile tra i dispositivi.
- Sensori SHT35 collegati via I2C (opzionale: usare `--simulate` per test senza sensore).
- LED (o relè simulato) collegato al `purple` Pi (pin configurato nello script).
- Python 3, `pip` e `virtualenv`.
- Mosquitto broker installato e in esecuzione sul `black` (172.16.32.182).
- SSH access ai Pi: `ssh pi@172.16.33.37` (purple), `ssh pi@172.16.33.38` (red), `ssh pi@172.16.32.182` (black).

## 2) Preparare il server (black) — Mosquitto e `server.py`
Esegui questi comandi sul `black` (172.16.32.182).

In [ ]:
# Comandi da eseguire sul server (black)
sudo apt update
sudo apt install -y mosquitto mosquitto-clients sqlite3
# Abilita mosquitto al boot
sudo systemctl enable --now mosquitto

# Posizionati nella cartella del progetto dove hai copiato gli script
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt

# Avvia il server.py (salverà i dati in temperatures.db)
python3 server.py --broker 172.16.32.182 --threshold 22.0

### Test rapidi sul server:
- Verificare i messaggi sensore: `mosquitto_sub -h 172.16.32.182 -t 'sensors/zone/+/temperature' -v`
- Vedere i comandi LED pubblicati: `mosquitto_sub -h 172.16.32.182 -t 'actuators/zone/purple/led' -v`
- Controllare il DB SQLite: `sqlite3 temperatures.db 'SELECT id,zone,temperature,timestamp FROM readings ORDER BY id DESC LIMIT 20;'`

## 3) Preparare il client Red (172.16.33.38) — `red_client.py`
Esempio di comandi da eseguire sul RPi red. Se non hai il sensore disponibile usa `--simulate`.

In [ ]:
ssh pi@172.16.33.38
# Abilita I2C (se hai il sensore reale): sudo raspi-config -> Interfacing Options -> I2C
python3 -m venv venv
source venv/bin/activate
pip install -r /path/to/repo/requirements.txt
# Se non usi il sensore reale aggiungi --simulate
python3 red_client.py --broker 172.16.32.182 --interval 10 --simulate

## 4) Preparare il client Purple (172.16.33.37) — `purple_client.py`
Questo client pubblica la temperatura della zona purple e ascolta i comandi LED.

In [ ]:
ssh pi@172.16.33.37
# Abilita I2C se hai SHT35
# Installa RPi.GPIO solo se usi GPIO reale: sudo apt install -y python3-rpi.gpio
python3 -m venv venv
source venv/bin/activate
pip install -r /path/to/repo/requirements.txt
# Avvio (simulazione consigliata per test iniziale)
python3 purple_client.py --broker 172.16.32.182 --interval 10 --simulate --led-pin 17

## 5) Testing e validazione
1. Avvia `server.py` sul black.
2. Avvia `red_client.py` e `purple_client.py`.
3. Sul server, esegui: `mosquitto_sub -h 172.16.32.182 -t 'sensors/zone/+/temperature' -v` per vedere i messaggi.
4. Se la temperatura di `red` o `purple` supera la soglia impostata (`--threshold`), il server pubblicherà `ON` su `actuators/zone/purple/led`. Altrimenti pubblicherà `OFF`.
5. Verifica che il `purple` client riceva il comando e cambi stato LED (o stampi il cambiamento in simulazione).

## 6) (Opzionale) Esempio unit systemd per eseguire gli script come servizio
Creare file `/etc/systemd/system/mqtt-server.service` con il contenuto d'esempio e abilitarlo.

In [ ]:
# Esempio (server) - salvare come /etc/systemd/system/mqtt-server.service (modificare i percorsi)
[Unit]
Description=MQTT Server script
After=network.target

[Service]
Type=simple
User=pi
WorkingDirectory=/home/pi/path/to/repo
ExecStart=/home/pi/path/to/repo/venv/bin/python /home/pi/path/to/repo/server.py --broker 172.16.32.182 --threshold 22.0
Restart=always

[Install]
WantedBy=multi-user.target

# Abilitare e avviare:
sudo systemctl daemon-reload
sudo systemctl enable --now mqtt-server.service

## 7) Pubblicare il repository su GitHub (passaggi PowerShell)
Di seguito trovi i comandi PowerShell da eseguire sul tuo PC/una macchina dove hai i file. Assicurati di aver creato un repository su GitHub (o usa `git remote add origin <url>`).
Se usi SSH, assicurati di avere la chiave SSH aggiunta a GitHub; se usi HTTPS potresti dover inserire token PAT per push sicuri.

In [ ]:
# Esempio PowerShell: inizializza repo, aggiungi remote, commit e push
git init
git add .
git commit -m "Initial commit: MQTT Multi-RPi setup"
# Aggiungi remote (esempio SSH):
git remote add origin git@github.com:USERNAME/REPO.git
git branch -M main
git push -u origin main

### .gitignore consigliato
- `venv/`
- `temperatures.db`
- `__pycache__/`
- `.vscode/`

## 8) Script PowerShell automatico `publish_to_github.ps1`
Lo script presente nella cartella del progetto semplifica: prende un parametro `-RemoteUrl` e opzionali `-Branch` e `-Message` e prova a inizializzare il repo, fare commit e push. Controlla che tu abbia configurato le credenziali Git/SSH.

In [ ]:
# Esempio di uso (PowerShell):
# .\publish_to_github.ps1 -RemoteUrl git@github.com:USERNAME/REPO.git -Branch main -Message "Initial commit from project"

## 9) Checklist finale (breve)
- [ ] Mosquitto in esecuzione su `black`.
- [ ] `server.py` avviato e salva letture in `temperatures.db`.
- [ ] `red_client.py` e `purple_client.py` in esecuzione (o in systemd).
- [ ] Verificato che, quando la temperatura supera la soglia, il topic LED riceve `ON`.
- [ ] Se desideri, push del repo su GitHub usando `publish_to_github.ps1` e controllo remoto.